In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import tensorflow as tf

np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

In [2]:
#csv파일 불러오기
train = pd.read_csv('/Users/keom/Desktop/work/Weather Forecast/train.csv')
test = pd.read_csv('/Users/keom/Desktop/work/Weather Forecast/test.csv')

In [3]:
#x칼럼 정리.
x_time = ['id'] #시간
x_temperature = ['X00', 'X07', 'X28', 'X31', 'X32'] #기온
x_local_pressure = ['X01', 'X06', 'X22', 'X27', 'X29'] #현지기압
x_wind_speed = ['X02', 'X03', 'X18', 'X24', 'X26'] #풍속
x_daily_rainfall = ['X04', 'X10', 'X21', 'X36', 'X39'] #일일 누적강수량
x_sealevel_pressure = ['X05', 'X08', 'X09', 'X23', 'X33'] #해면기압
x_daily_sun = ['X11', 'X34'] #일일 누적일사량 (X14, X16, X19는 제거됨)
x_humidity = ['X12', 'X20', 'X30', 'X37', 'X38'] #습도
x_wind_direction = ['X13', 'X15', 'X17', 'X25', 'X35'] #풍향

#y칼럼 정리
y_sensor = ['Y00', 'Y01','Y02','Y03','Y04', 'Y05', 'Y06', 'Y07', 'Y08', 
            'Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17'] #센서측정온도
y_target = ['Y18'] #예측대상

In [4]:
def accumulate_to_10minute(df, col):
    for i in col:
        #누적데이터를 10분간의 데이터로 분류
        tmp = df[i].iloc[0]
        df[i] = df[i] - df[i].shift(1)
        df[i].iloc[0] = tmp
        
        #24시를 넘어서 데이터가 음수가 된다면, 0으로 변경
        for j in np.arange(0,len(df[i])):
            if df[i].iloc[j] < 0:
                df[i].iloc[j] = 0
    print("누적 데이터 변경 완료")


def remove_outlier(df):
    for j in list(df.columns):
        for i in np.arange(0,len(df["X00"])):
            if df.loc[i,j] > 3:
                df.loc[i,j] = df.shift(1).loc[i,j]

In [5]:
#고장난 센서 X14, X16, X19 삭제
train = train.drop(['X14', 'X16', 'X19'], axis =1)
test = test.drop(['X14', 'X16', 'X19'], axis =1)
print('X14, X16, X19 삭제 완료')


#누적 강수량 변경
accumulate_to_10minute(train, x_daily_rainfall)
accumulate_to_10minute(test, x_daily_rainfall)


#누적 일사량 변경
accumulate_to_10minute(train, x_daily_sun)
accumulate_to_10minute(test, x_daily_sun)

X14, X16, X19 삭제 완료
누적 데이터 변경 완료


/opt/anaconda3/envs/venv/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


누적 데이터 변경 완료
누적 데이터 변경 완료
누적 데이터 변경 완료


In [6]:
#학습용 데이터프레임 생성
train_df = pd.DataFrame(pd.concat([train[x_temperature], 
                                  train[x_daily_sun]], axis=1))

test_df = pd.DataFrame(pd.concat([test[x_temperature], 
                                 test[x_daily_sun]], axis=1))

In [7]:
# standardization을 위해 평균과 표준편차 구하기
MEAN = train_df.mean()
STD = train_df.std()


# 표준편차가 0일 경우 대비하여 1e-07 추가 
train_df = (train_df - MEAN) / (STD + 1e-07)

test_df = (test_df - MEAN) / (STD + 1e-07)

In [8]:

remove_outlier(train_df)
remove_outlier(test_df)


In [9]:
def convert_to_timeseries(df, interval):
    sequence_list = []
    target_list = []
    
    for i in tqdm(range(df.shape[0] - interval)):
        sequence_list.append(np.array(df.iloc[i:i+interval,:-1]))
        target_list.append(df.iloc[i+interval,-1])
    
    sequence = np.array(sequence_list)
    target = np.array(target_list)
    a
    return sequence, target

In [13]:
train_df

,X00,X07,X28,X31,X32,X11,X34
0,-2.236377,-2.160752,-2.314454,-2.109779,-1.766623,-0.794399,-0.762928
1,-2.321347,-2.183598,-2.361789,-2.091664,-1.842501,-0.794399,-0.762928
2,-2.300105,-2.183598,-2.361789,-2.146009,-1.956318,-0.794399,-0.762928
3,-2.300105,-2.206443,-2.361789,-2.200355,-2.013226,-0.794399,-0.762928
4,-2.342589,-2.206443,-2.385457,-2.254700,-2.013226,-0.794399,-0.762928
...,...,...,...,...,...,...,...
4747,-0.069655,0.009577,0.407308,-0.388845,-0.173188,-0.794399,-0.762928
4748,-0.069655,-0.013269,0.359973,-0.388845,-0.154219,-0.794399,-0.762928
4749,-0.112139,-0.058960,0.359973,-0.370730,-0.154219,-0.794399,-0.762928
4750,-0.175867,-0.081806,0.288971,-0.388845,-0.135249,-0.794399,-0.762928


In [10]:
x_num = 7


#학습에 사용할 Y값 지정
y_columns = ['Y09', 'Y16','Y15']


#지정한 Y들만큼 데이터의 길이를 늘림
sequence = np.empty((0, 12, x_num))
target = np.empty((0,))

for column in y_columns :
    
    concat = pd.concat([train_df, train[column]], axis = 1)

    _sequence, _target = convert_to_timeseries(concat.head(144*30), interval = 12)

    sequence = np.vstack((sequence, _sequence))
    target = np.hstack((target, _target))

100%|██████████| 4308/4308 [00:00<00:00, 5285.63it/s]


In [12]:
sequence.shape

(12924, 12, 7)

In [14]:
target.shape

(12924,)

In [52]:
"""
#6. test_df 생성 (x변수를 평균 사용)
"""

train_df['dummy'] = 0
test_df['dummy'] = 0

X_test, _ = convert_to_timeseries(pd.concat([train_df, test_df], axis = 0), interval=12)
X_test = X_test[-11520:, :, :]

# 만들어 두었던 dummy feature 제거
train_df.drop('dummy', axis = 1, inplace = True)
test_df.drop('dummy', axis = 1, inplace = True)

100%|██████████| 16260/16260 [00:07<00:00, 2273.28it/s]


In [53]:

"""

#7. 학습 모델 생성

"""


simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, input_shape=sequence.shape[-2:]),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mse')
simple_lstm_model.summary()


# loss가 4미만으로 떨어지면 학습 종료 시키는 기능
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if(logs.get('loss') < 4):
            print('\n Loss is under 4, cancelling training')
            self.model.stop_training = True
            

callbacks = myCallback()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               69632     
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 143,873
Trainable params: 143,873
Non-trainable params: 0
_________________________________________________________________


In [54]:

"""
#8. 모델 학습
"""
simple_lstm_model.fit(
    sequence, target,
    epochs=60,
    batch_size=64,
    verbose=1,
    shuffle=False,
    callbacks = [callbacks])

Train on 12924 samples
Epoch 1/60
12924/12924 [==============================] - 3s 220us/sample - loss: 57.7122
Epoch 2/60
12924/12924 [==============================] - 2s 132us/sample - loss: 5.4848
Epoch 3/60
12924/12924 [==============================] - 2s 132us/sample - loss: 4.7786
Epoch 4/60
12924/12924 [==============================] - 2s 133us/sample - loss: 4.7025
Epoch 5/60
12924/12924 [==============================] - 2s 133us/sample - loss: 4.5391
Epoch 6/60
12924/12924 [==============================] - 2s 133us/sample - loss: 4.3452
Epoch 7/60
12924/12924 [==============================] - 2s 132us/sample - loss: 4.2454
Epoch 8/60
12924/12924 [==============================] - 2s 132us/sample - loss: 4.1983
Epoch 9/60
12924/12924 [==============================] - 2s 132us/sample - loss: 4.1519
Epoch 10/60
12924/12924 [==============================] - 2s 132us/sample - loss: 4.1366
Epoch 11/60
12924/12924 [==============================] - 2s 133us/sample - loss: 4.

In [55]:
"""

#9. 마지막 3일 데이터('Y18')를 사용하여 fine-tuning 

"""

simple_lstm_model.layers[0].trainable = False
finetune_X, finetune_y = convert_to_timeseries(pd.concat([train_df.tail(432), 
                                                          train['Y18'].tail(432)], axis = 1), 
                                               interval=12)


finetune_history = simple_lstm_model.fit(
            finetune_X, finetune_y,
            epochs=32,
            batch_size=64,
            shuffle=False,
            verbose = 1)

100%|██████████| 420/420 [00:00<00:00, 2702.63it/s]

Train on 420 samples
Epoch 1/32
 64/420 [===>..........................] - ETA: 0s - loss: 8.1879

420/420 [==============================] - 0s 162us/sample - loss: 5.9828
Epoch 2/32
420/420 [==============================] - 0s 170us/sample - loss: 8.8941
Epoch 3/32
420/420 [==============================] - 0s 169us/sample - loss: 5.1371
Epoch 4/32
420/420 [==============================] - 0s 161us/sample - loss: 6.0095
Epoch 5/32
420/420 [==============================] - 0s 157us/sample - loss: 4.8506
Epoch 6/32
420/420 [==============================] - 0s 153us/sample - loss: 4.7099
Epoch 7/32
420/420 [==============================] - 0s 173us/sample - loss: 4.2745
Epoch 8/32
420/420 [==============================] - 0s 164us/sample - loss: 5.2603
Epoch 9/32
420/420 [==============================] - 0s 158us/sample - loss: 3.5021
Epoch 10/32
420/420 [==============================] - 0s 162us/sample - loss: 5.0034
Epoch 11/32
420/420 [==============================] - 0s 159us/sample - loss: 3.3138
Epoch 12/32
420/420 [==============================] - 0s 158us/sample - l

In [56]:
#10. 결과 예측하기 
finetune_pred = simple_lstm_model.predict(X_test)


# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':finetune_pred.reshape(1,-1)[0]})

submit.to_csv('/Users/keom/Desktop/submit3.csv', index = False)